In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
id = '79300479'
url = f'https://fbref.com/en/players/{id}/matchlogs/2024-2025/Martin-Odegaard-Match-Logs'

In [ ]:
html = requests.get(url)

soup = BeautifulSoup(html.text, 'html.parser')

In [ ]:
# Locate the table
table = soup.find("table", id="matchlogs_all")

# Extract rows
rows = table.find_all("tr")

# Extract headers
headers = [header.text for header in rows[0].find_all("th")]

# Extract data
data = []
for row in rows[1:]:
    cells = row.find_all("td")
    data.append([cell.text for cell in cells])

# Display results
print("Headers:", headers)
print("Data:", data)



In [3]:
print(len(str('e4b73855-cec2-3638-b498-06d55ee89c2a')))

36


In [2]:
print(len([1,2,3]))

3


In [1]:
print('bc')

bc
